In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

# Caricamento del dataset MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizzazione dei dati e preparazione
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Creazione del modello
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compilazione del modello
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Addestramento del modello
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Valutazione del modello
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")

# Salvataggio del modello
model.save('/app/mnist_model.keras')
print("Modello salvato come 'mnist_model.keras'")

2025-01-19 02:12:44.494374: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-19 02:12:44.577060: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-19 02:12:45.321811: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-19 02:12:46.031129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737252766.568708    1270 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737252766.75

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/app/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1737252792.568340    1270 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8972 - loss: 0.3396 - val_accuracy: 0.9793 - val_loss: 0.0673
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9841 - loss: 0.0518 - val_accuracy: 0.9866 - val_loss: 0.0449
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9902 - loss: 0.0326 - val_accuracy: 0.9880 - val_loss: 0.0438
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9930 - loss: 0.0206 - val_accuracy: 0.9870 - val_loss: 0.0422
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9945 - loss: 0.0159 - val_accuracy: 0.9872 - val_loss: 0.0479
313/313 - 1s - 3ms/step - accuracy: 0.9898 - loss: 0.0318
Test accuracy: 0.989799976348877
Modello salvato come 'mnist_model.keras'


In [2]:
from aima.search import Problem, astar_search, breadth_first_tree_search
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Definizione della classe UniformColoring che estende Problem
class UniformColoring(Problem):
    def __init__(self, initial, goal):
        super().__init__(initial, goal)

    def actions(self, state):
        actions = []
        agent_pos = state['agent_pos']
        grid = state['grid']
        max_row, max_col = grid.shape

        # Movimento
        moves = {'Up': (-1, 0), 'Down': (1, 0), 'Left': (0, -1), 'Right': (0, 1)}
        for action, (dr, dc) in moves.items():
            new_row = agent_pos[0] + dr
            new_col = agent_pos[1] + dc
            if 0 <= new_row < max_row and 0 <= new_col < max_col:
                actions.append(action)

        # Colorazione (se il colore della cella corrente non è quello desiderato)
        current_color = state['grid'][agent_pos]
        goal_color = self.goal['grid'][agent_pos]
        if current_color != goal_color:
            actions.append('Paint')

        return actions

    def result(self, state, action):
        new_state = {
            'agent_pos': state['agent_pos'],
            'grid': state['grid'].copy()
        }

        if action == 'Up':
            new_state['agent_pos'] = (state['agent_pos'][0] - 1, state['agent_pos'][1])
        elif action == 'Down':
            new_state['agent_pos'] = (state['agent_pos'][0] + 1, state['agent_pos'][1])
        elif action == 'Left':
            new_state['agent_pos'] = (state['agent_pos'][0], state['agent_pos'][1] - 1)
        elif action == 'Right':
            new_state['agent_pos'] = (state['agent_pos'][0], state['agent_pos'][1] + 1)
        elif action == 'Paint':
            agent_pos = state['agent_pos']
            new_state['grid'][agent_pos] = self.goal['grid'][agent_pos]

        return new_state

    def goal_test(self, state):
        return np.array_equal(state['grid'], self.goal['grid'])

    def h(self, node):
        state = node.state
        mismatched_cells = np.sum(state['grid'] != self.goal['grid'])
        agent_pos = state['agent_pos']
        goal_positions = np.argwhere(state['grid'] != self.goal['grid'])
        if goal_positions.size == 0:
            return 0
        distances = [abs(agent_pos[0] - pos[0]) + abs(agent_pos[1] - pos[1]) for pos in goal_positions]
        return mismatched_cells + min(distances)

# Caricamento dell'immagine di input
image = cv2.imread('/app/griglia_input.png', cv2.IMREAD_GRAYSCALE)

# Pre-elaborazione dell'immagine
_, thresh = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV)
contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cells = []
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if w > 20 and h > 20:
        cell = image[y:y+h, x:x+w]
        cells.append((cell, (x, y, w, h)))

# Ordinamento delle celle
cells.sort(key=lambda x: (x[1][1], x[1][0]))
grid_size = int(np.sqrt(len(cells)))
grid = np.zeros((grid_size, grid_size), dtype=int)
agent_pos = None

# Caricamento del modello pre-addestrato per la classificazione
model = load_model('/app/mnist_model.keras')

# Classificazione delle celle
index = 0
for i in range(grid_size):
    for j in range(grid_size):
        cell_image, (x, y, w, h) = cells[index]
        cell_image = cv2.resize(cell_image, (28, 28))
        cell_image = cell_image.reshape(1, 28, 28, 1) / 255.0
        prediction = model.predict(cell_image)
        predicted_class = np.argmax(prediction)

        if predicted_class >= 10:  # Supponendo che le lettere abbiano classi >=10
            agent_pos = (i, j)
            grid[i, j] = 0
        else:
            grid[i, j] = predicted_class  # Colore della cella

        index += 1

# Definizione dello stato iniziale e goal
initial_state = {
    'agent_pos': agent_pos,
    'grid': grid
}

goal_grid = np.zeros_like(grid)  # Ad esempio, tutte le celle devono essere 0
goal_state = {
    'agent_pos': agent_pos,  # L'agente può essere ovunque alla fine
    'grid': goal_grid
}

# Creazione del problema e risoluzione con A*
problem = UniformColoring(initial_state, goal_state)
solution_node = astar_search(problem)

# Estrazione delle azioni dalla soluzione
actions = solution_node.solution()
print("Azioni trovate con A* Search:")
print(actions)

# Simulazione dell'esecuzione
def simulate_execution(initial_state, actions):
    state = initial_state.copy()
    frames = []

    for action in actions:
        state = problem.result(state, action)
        grid_display = state['grid'].copy()
        agent_pos = state['agent_pos']
        grid_display[agent_pos] = -1  # Codice per visualizzare l'agente

        frames.append(grid_display)

    return frames

frames = simulate_execution(initial_state, actions)

# Visualizzazione della simulazione
for i, frame in enumerate(frames):
    plt.imshow(frame, cmap='viridis')
    plt.title(f'Passo {i+1}')
    plt.show()

Azioni trovate con A* Search:
[]
